In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bts
import requests as req
import asyncio
from tqdm import tqdm
from typing import List, Tuple, Union
import re

In [2]:
with open('html/intro_naves.html','r',encoding='UTF-8') as file:
    contenido=file.read()
soup=bts(contenido,'html.parser')
href=[a.get('href').replace('..','https://space.skyrocket.de') for a in soup.find_all('table')[0].find_all('a') if int(a.text) > 1950 and int(a.text) < 2023]

In [3]:
def asincrono(funcion):
    ''' esto es un decorador que permite colocarlo encima de una funcion para poder ejecutar otras mientras se espera el resultado'''
    def eventos(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, funcion, *args, **kwargs)
    
    return eventos

In [4]:
def DataFrame_table(url) -> pd.DataFrame:
    '''ulr contenido de la web que se va a transformar:
    DataFrame_table(url)
    ulr-> el contenido que va a leer y transformar
    return -> dataframe'''
    response=req.get(url)
    html_content=response.text
    soup=bts(html_content,'html.parser')

    table=soup.find('table',{'class':'data'})
    content_table=[content.text for content in table.find_all('td')]
    keys=[e.text for e in table.find('tr',{'class':'bigheadline'}) if e.text != '\n']
    values=[content_table[e:e+6] for e in range(0,len(content_table),6)]
    
    ref = soup.find('div', {'class': 'container'}).text.split('\n')
    corte1=soup.find('div',{'class': 'container'}).text.split('\n').index('Launch sites:')+1
    corte2=soup.find('div', {'class': 'container'}).text.split('\n').index('\t\r')-1
    colum_ref={e.split('=')[0].strip() : e.split('=')[-1].strip() for e in ref[corte1:corte2]}

    table_dic={key : [] for key in keys}


    for sublist in values:
        for a in range(6):
            table_dic[keys[a]].append(sublist[a])

    df=pd.DataFrame(table_dic)

    def fail(x) -> str:
        '''Simplifica los valores de las descripciones dadas como : fallo ,exito , tipo de vuelo y dudoso
        x -> es la str que entra para comprobar los parametros'''
        phrase=x.lower()

        if 'fail' in phrase:
            x='Failed'
            return x
        elif any(word in phrase for word in ['explo','wrong','damaged','abort','not deployed','error','Lower']):
            return 'Failed'
        elif any(word in phrase for word in ['1st','first flight']):
            return 'First flight'
        elif 'last flight' in phrase:
            return'Last flight'
        elif 'note' in phrase:
            return x
        else:
            return 'Pass'
    
    def complate(x) -> str:
        '''sustituye valores vacios como exitosos
        x -> es la str que se lee'''
        if x =='':
            x='Pass'
            return x
        else:
            return x
        
    def clean_df(df) -> pd.DataFrame:  
        ''' se encarga de ordenar y limpiar el df
        df -> dataframe que desea pasar por la limpieza'''
        df['Date']=df['Date'].apply(lambda x : x.replace('?','') if '?' in x else x)
        df['Date']=df['Date'].astype('datetime64[ns]')
        df['Remark']=df['Remark'].apply(complate)
        df['Payload(s)']=df['Payload(s)'].str.replace('\n',',')
        df['Remark']=df['Remark'].apply(fail)

        Succes=df['Remark']

        df_change=df.drop(columns='Remark')
        df_change['Site_inf']=df_change['Site'].apply(lambda x : x.split()[-1])
        df_change['Site']=df_change['Site'].apply(lambda x : x.split()[0])
        df_change['complate_site']=df_change['Site'].map(colum_ref)
        
        complete_df=pd.concat([df_change,Succes],axis=1)

        return complete_df
    
    df_clean=clean_df(df)

    return df_clean

@asincrono
def data(href) -> pd.DataFrame:
    '''a partir de una lista de url se crea un df :

    misiones=pd.DataFrame()

    for i in range(len(href)): -> lee la longitud de la lista para extraer df de manera individual 
    
        try:
            misiones=pd.concat([misiones,DataFrame_table(href[i])],axis=0) ->concatena el df con el principal para crear un dataframe unico 

        except Exception as e: -> en caso de querer sasber el fallo
            print(f'{e}:{i}')
            continue
        '''
    misiones=pd.DataFrame()
    for i in range(len(href)):
        try:
            misiones=pd.concat([misiones,DataFrame_table(href[i])],axis=0)
        except Exception as e:
            print(f'{e}:{i}')
            continue
    return misiones

In [5]:
def get_url(url)->list:
    '''entra una lista de urls/href que buscara las urls internas de lanzamientos
    devuelve una lista de urls de una pagina
    get_url() -> ejemplo : 
    1) insertar una unica url_get('https://aqui.web/pones/tu/url')
    return -> lista de urls que hay en una pagina web
    '''
    response=req.get(url)
    html_content=response.text
    soup=bts(html_content,'html.parser')

    table=soup.find('table',{'class':'data'})
    content_table=[content for content in table.find_all('td')]
    values=[content_table[e:e+6] for e in range(0,len(content_table),6)]
    filtered_values = [[td for td in sublist if any(a in td for a in td if a.name == 'a')] for sublist in values]
    launch_veicles=[]
    
    for a in filtered_values:
        url=a[-1].find('a').get('href').replace('..','https://space.skyrocket.de')
        launch_veicles.append(url)
    return launch_veicles

@asincrono
def url_launches(url) -> list:
    '''entra una lista de urls/href que buscara las urls internas de lanzamientos
    devuelve una lista dee urls de todas las paginas
    url_launches(url) -> se usa una lista de urls para podedr ejecutar la funcioin : get_url
    y lo guarda en una lista de valores unicos

    urls=[]  -> valores unicos 
    
    for i in url:
    
        list_url_page=get_url(i)
        for new_url in list_url_page:
            if new_url not in urls:
                urls.append(new_url)

    return urls -> devuelve lista de valores unicos'''
    urls=[]
    for i in url:
        list_url_page=get_url(i)
        for new_url in list_url_page:
            if new_url not in urls:
                urls.append(new_url)
    return urls

In [6]:
def rockets(url) -> pd.DataFrame:

    response=req.get(url)
    html_content=response.text
    soup=bts(html_content,'html.parser')

    try:
        '''hay dos tipos de tablas que me interesan porlo que si da error en una es la otra'''
        info=soup.find('table',{'id':'rocdata'}).find_all('tr')
        keys=[document.text for document in info[0] if document.text != '\n']

        try:
            '''hay dos tipos de diccionarios y para evitar errores pongo ambas cualidades'''
            values=[document.text for document in info[-1] if document.text != '\n']
            table_rocket={keys[e] : [values[e]] for e in range(len(keys))}

            df=pd.DataFrame(table_rocket)
            colum_launch=pd.DataFrame({'Launch Vehicle':[url.split('/')[-1].split('.')[0]]})
            final_df=pd.concat([colum_launch,df],axis=1)
        except:
            values=[e.strip() for e in [e.text for e in info[-1] if e.text!='\n'][0].split('/')]
            table_rocket={keys[e] : [values[e]] for e in range(len(keys))}

            df=pd.DataFrame(table_rocket)
            colum_launch=pd.DataFrame({'Launch Vehicle':[url.split('/')[-1].split('.')[0]]})
            final_df=pd.concat([colum_launch,df],axis=1)
        
    except:
        
        info=soup.find('table',{'id':'rocperf'}).find_all('tr')
        keys=[document.text for document in info[0] if document.text != '\n']
        values=[document.text for document in info[-1] if document.text != '\n']
        table_rocket={keys[e] : [values[e]] for e in range(len(keys))}

        df=pd.DataFrame(table_rocket)
        colum_launch=pd.DataFrame({'Launch Vehicle':[url.split('/')[-1].split('.')[0]]})
        final_df=pd.concat([colum_launch,df],axis=1)
        
    return final_df

def rockets_table(href) -> pd.DataFrame:

    final_df=pd.DataFrame()
    other_df=pd.DataFrame()
    for url in href:
        try:
            df=rockets(url)
            if 'LEO' not in list(df.columns):
                final_df=pd.concat([final_df,df],axis=0)
            else:
                other_df=pd.concat([other_df,df],axis=0)
        except:
            continue
    
    other_df=other_df[other_df.columns.to_list()[:3]]
    final_df=final_df[final_df.columns.to_list()[:-4]]
    
    return final_df,other_df

def lists_of_launches(href) -> tuple:

        '''
        lists_of_launches(href) => aqui se aplica la funcion: result1 = lists_of_launches('launch1', 'launch2', 'launch3')
        o launches = ['launch4', 'launch5', 'launch6']
        return => devuelve dos argumentos'''
        
        response = req.get(href)
        response_html=response.text
        soup = bts(response_html,'html.parser')
        table = soup.find('div',{'class':'llist'}).find_all('pre')
        clave_html = [e.find_all('a') for e in table if e.find_all('a')][0]
        keys = [e.text for e in clave_html]
        dicio_clave = {e.text:e.get('href').replace('..','https://space.skyrocket.de') for e in clave_html}

        lista_tablas = []
        lista_url_sat = [url for key,url in dicio_clave.items()]

        for content in table:
                for line in content.text.splitlines():
                        if any(key in line for key in keys):
                                lista_tablas.append(line)

        result=(lista_tablas,lista_url_sat)

        return result

def general_fun_of_lists(lists,fun) -> Union[list, Tuple]:
    '''
    general_fun_of_lists (lists,fun) => es una funcion que se encarga de iterar lista en las funciones que uno necesita
    litst => [una,lista,de,elementos,que,desea,trabajar]
    fun => la funcion que se va ha aplicar en cada elemento 
    return => depende de los resultados de la funcion que se use:

    1) da una lista unica => return solucion
    2) da una lista de dos resultados => return (solucion,solucion2)

    opcional: errores

    0) en caso de que haya errores esta funcion te enseña donde ha pasado, y el error:
       por eso en este caso puede ocurrir dos cosas:
       1)si la lista unica tiene errores, te devolvera una tupla donde el primero es la solucion y la segundd los errores => return (solucion, errores)
       2)si es una tupla de dos soluciones, te devuelve las soluciones mas el error => return (solucion,solucion2,errores)

    ejemplo:

    def encontrar_x() -> lista          fun de ejemplo

    general_fun_of_lists([una,lista,de,elementos,que,desea,trabajar],encontrar_x)
    '''
    sol = []
    sol_2 = []
    error = []

    for element in lists:

        try:
            result = fun(element)
            if len (result) == 1:
                sol.extend(result)
            elif len (result) == 2:
                sol.extend(result[0])
                sol_2.extend(result[-1])
        except Exception as e:
            error.append(f'{e}:{element}')

    if sol_2:
        if error:
            result = (sol,sol_2,error)
            return result
        else:
            result = (sol,sol_2)
            return result
    elif sol:
        if error:
            result = (sol,error)
            return result
        else:
            return sol


In [7]:
def visualizar_fallos(urls,fun):
    '''
    uso en caso de querer descubrir donde esta fallando la funcion:
    visualizar_fallos(urls,fun)
    urls -> urls que necesita leer la funcion para poner aprueba la peticion
    fun -> es la funcion que quieres testear para su funcionalidad
    return -> devuelve uns lista de errores
    '''
    errors=[]

    for i in range(len(urls)):
        try:
            fun(urls[i])
        except Exception as e:
            errors.append(f'{e}:{i}')
            
    
    return errors

In [8]:
help(visualizar_fallos)

Help on function visualizar_fallos in module __main__:

visualizar_fallos(urls, fun)
    uso en caso de querer descubrir donde esta fallando la funcion:
    visualizar_fallos(urls,fun)
    urls -> urls que necesita leer la funcion para poner aprueba la peticion
    fun -> es la funcion que quieres testear para su funcionalidad
    return -> devuelve uns lista de errores



In [9]:
misiones=data(href)
launches=url_launches(href)

In [10]:
misiones=misiones.result()
launches_list=launches.result()

In [11]:
rockes_1,rockes_2=rockets_table(launches_list)

In [17]:
rockes_1

,Launch Vehicle,Version,Strap-On,Stage 1,Stage 2,Stage 3
0,sputnik-1,Sputnik (8K71PS),"Blok-B,V,G,D / 4 × RD-107-8D74PS",Blok-A / RD-108-8D75PS,NaN,NaN
0,vanguard,Vanguard,NaN,X-405,AJ-10-37,GRC 133-KS-2800
0,juno-1,Juno-1,NaN,Redstone / A-7,11 × Baby Sergeant,3 × Baby Sergeant
0,sputnik-2,Sputnik (8A91),"Blok-B,V,G,D / 4 × RD-107-8D76",Blok-A / RD-108-8D77,NaN,NaN
0,pilot,Pilot (NOTS-EV1H),NaN,2 × 2 × HOTROC,X-241,NOTS-100
...,...,...,...,...,...,...
0,sslv,SSLV,NaN,S-85,S-7,S-4
0,falcon-9-heavy_b5_px,Falcon-Heavy (Block 5) (px),"2 × Stage 1 (str., reusable) / 9 × Merlin-1D",Stage 1 (str.) / 9 × Merlin-1D (upr.+),Stage 2 (str.) / Merlin-1D-Vac (upr.+),NaN
0,sls_bl1_icps,SLS (Block-1) iCPS,2 × RSRM-5,4 × RS-25D,iCPS / RL10B-2,NaN
0,jielong-3,Jielong-3 (Smart Dragon-3),NaN,?,?,?


In [18]:
rockes_2

,Launch Vehicle,Performance (kg),LEO
0,atlas-d_mercury,Atlas-D Mercury,1350
0,atlas-slv3,Atlas-SLV3,800
0,slv-3,SLV-3,40


In [12]:
resultado=general_fun_of_lists(launches_list,lists_of_launches)

In [13]:
lista_launches_trabajar,urls_sat,errores=resultado

In [14]:
clean=[[element for element in file.split('  ') if element != ''] for file in lista_launches_trabajar]

#RECONTRUCCION DE LOS DATOS 

In [15]:
lenght_of_info=set([len(e) for e in clean])
dicio_of_launches_info={e : [] for e in lenght_of_info}
for line in clean:
    dicio_of_launches_info[len(line)].append(line)
dicio_of_launches_info

dataframes=[e for e in lenght_of_info]
for element in lenght_of_info:
    dataframes[element-1]=pd.DataFrame(dicio_of_launches_info[element])

In [16]:
keys=[element.split('/')[-1].split('.')[0] for element in launches_list]

In [17]:
df_8=dataframes[8]
pattern = r'^\d{2}\.\d{2}\.\d{4}$'

In [18]:
df_8[4]=df_8[4].str.replace(' ','')

for column in df_8.columns:
    df_8[column]=df_8[column].str.strip()

for column in df_8.columns:
    df_8.loc[df_8[column].str.contains(pattern), 'date'] = df_8.loc[df_8[column].str.contains(pattern), column]

for column in df_8.columns:
    df_8.loc[df_8[column].str.lower().isin(keys), 'type'] = df_8.loc[df_8[column].str.lower().isin(keys), column]

for column in df_8.columns:  
    df_8.loc[df_8[column].str.contains(r'^[A-Z0-9-/]{2,}$') , 'serial'] = df_8.loc[df_8[column].str.contains(r'^[A-Z0-9-/]{2,}$') , column]

for column in df_8.columns: 
    df_8.loc[df_8[column].str.contains(r'^[A-Za-z0-9\\\-/]+ [A-Za-z0-9\\\-/]{4,}$'), 'ls'] =  df_8.loc[df_8[column].str.contains(r'^[A-Za-z0-9\\\-/]+ [A-Za-z0-9\\\-/]{4,}$')  , column]

df_8['payload']=df_8[8]

for column in df_8.columns: 
    df_8.loc[df_8[column].str.contains(r'^(P|F)$'), 'succes'] =  df_8.loc[df_8[column].str.contains(r'^(P|F)$')  , column]

df_8['succes'] = df_8['succes'].fillna('Ok')

clean_8 = df_8[df_8.columns[-6:]]

c:\Users\sebas\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [19]:
clean_8

,date,type,serial,ls,payload,succes
0,19.10.1961,Scout-X1,ST-7,WI LA-3,P 21,Ok
1,29.03.1962,Scout-X2,ST-9,WI LA-3,P 21A,Ok
2,22.05.1963,Scout-X3,116,WI LA-3,RFD 1,Ok
3,20.07.1964,Scout-X4,124R,WI LA-3A,SERT 1,Ok
4,20.09.1971,Scout-B,BIC,WI LA-3A,BIC,Ok
5,18.06.1976,Scout-D1,193C,WI LA-3A,GP A (Gravity Probe 1),Ok
6,12.07.2001,Ariane-5G,V-142/510,Ko ELA-3,Artemis / BSat 2b,P
7,11.12.2002,Ariane-5ECA,V-157/517,Ko ELA-3,Hotbird 7 / Stentor / MFD A / MFD B,F
8,25.01.2018,Ariane-5ECA,VA-241/5101,Ko ELA-3,SES 14/GOLD / Al Yah 3,P
9,25.05.2020,LauncherOne,B-747-400,Mo RW12/30,Starshine 4 / Intern-Sat,F


In [20]:
def buscador_links(nombre_lanzamiento):
    for e in launches_list:
        if nombre_lanzamiento in e:
            print(e)

In [21]:
def buscador_links_sat(nombre_lanzamiento):
    for e in urls_sat:
        if nombre_lanzamiento in e:
            print(e)

In [22]:
df_7 = dataframes[7]

In [23]:
pattern = r'^(?:\d{2}\.\d{2}\.\d{4}(?: [A-Z])?|(?:[A-Z] )?\d{2}\.\d{2}\.\d{4}|(?:\d{2}\.\d{2}\.\d{4}\.)|\d{2}\.\d{2}\.\d{4})(?: [a-z])?(?: \.)?$'
pattern_serial = r'^(?![0-9]{0,2}$)([A-Z0-9-/]{3,}|[A-Z0-9-\s)(]{2,}\([A-Z]{2}\)|[A-Za-z]{2}[0-9]{5}\-[0-9]{3}|[A-Z0-9/.]+\.[0-9/.])$'
pattern_ls= r'^(?:[A-Z]{2}|[A-Za-z]{2}\s[A-Za-z0-9-/]+|[A-Z]{2})$'

In [24]:
for column in df_7.columns:
    df_7[column]=df_7[column].str.strip()

In [125]:
df_7.drop(columns = 'date', inplace = True)

In [126]:
for column in df_7.columns:
    df_7.loc[df_7[column].str.contains(pattern), 'date'] = df_7.loc[df_7[column].str.contains(pattern), column]

df_7['date'] = df_7['date'].fillna('00.00.0000')

In [26]:
for column in df_7.columns:
    df_7.loc[df_7[column].str.contains(pattern), 'date'] = df_7.loc[df_7[column].str.contains(pattern), column]

df_7['date'] = df_7['date'].fillna('01.10.0001')

df_7[4] = df_7[4].str.replace(' ','')

for column in df_7.columns:  
    df_7.loc[df_7[column].str.contains(pattern_serial) , 'serial'] = df_7.loc[df_7[column].str.contains(pattern_serial) , column]

df_7['serial'] = df_7['serial'].fillna('NoSeial')

c:\Users\sebas\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [489]:
for column in df_7.columns:
    df_7.loc[df_7[column].str.lower().isin(keys), 'type'] = df_7.loc[df_7[column].str.lower().isin(keys), column]

In [31]:
for column in df_7.columns: 
    df_7.loc[df_7[column].str.contains(pattern_ls), 'ls'] =  df_7.loc[df_7[column].str.contains(pattern_ls)  , column]

df_7[6] = df_7[6].apply(lambda x : '' if x=='ST' or x=='SF' or x=='SL' else x)

df_7['ls'] = df_7['ls'].fillna('des')

df_7['success'] = df_7[6].apply(lambda x: 'F' if any(letter in x for letter in ['F', 'P', 'p']) else '')

df_7.loc[df_7['date'].str.contains(r'[A-Za-z]$'), 'success'] = 'F'

df_7['success'] = df_7['success'].apply(lambda x : 'Ok' if x == '' else x)

df_7['date']=df_7['date'].str.replace('F','').str.replace('P','').str.replace('p','')

df_7['date'] = df_7['date'].str.replace(' ','')

df_7['date'] = df_7['date'].apply(lambda x : x[:-1] if len(x) == 11 else x)

In [156]:
df_7

,0,1,2,3,4,5,6,7,serial,ls,payload,success,date
0,38,2,Juno-1,RS-26 (UV),05.03.1958,F,CC LC-26A,Explorer 2,RS-26 (UV),CC LC-26A,Explorer 2,Ok,05.03.1958
1,47,5,Juno-1,RS-47 (TI),24.08.1958,F,CC LC-5,Explorer 5,RS-47 (TI),CC LC-5,Explorer 5,Ok,24.08.1958
2,1,1,Vostok-L,B1-3,23.09.1958,F,Ba LC-1/5,Luna (1a) (Ye-1 1),B1-3,Ba LC-1/5,Luna (1a) (Ye-1 1),Ok,23.09.1958
3,2,2,Vostok-L,B1-4,11.10.1958,F,Ba LC-1/5,Luna (1b) (Ye-1 2),B1-4,Ba LC-1/5,Luna (1b) (Ye-1 2),Ok,11.10.1958
4,3,3,Vostok-L,B1-5,04.12.1958,F,Ba LC-1/5,Luna (1c) (Ye-1 3),B1-5,Ba LC-1/5,Luna (1c) (Ye-1 3),Ok,04.12.1958
...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,9,5,Astra Rocket-3.3,LV0010,12.06.2022,F,CC SLC-46,TROPICS 02 / TROPICS 04,LV0010,CC SLC-46,TROPICS 02 / TROPICS 04,Ok,12.06.2022
758,1,1,Firefly-Alpha,FLTA001,03.09.2021,Va SLC-2W,F,Serenity / Hiapo / BSS 1 / FossaSat 2 / FossaS...,FLTA001,Va SLC-2W,Serenity / Hiapo / BSS 1 / FossaSat 2 / FossaS...,F,03.09.2021
759,2,2,Firefly-Alpha,FLTA002,01.10.2022,Va SLC-2W,p,"Serenity (2) / TechEdSat 15 / GENESIS G, J / Q...",FLTA002,Va SLC-2W,"Serenity (2) / TechEdSat 15 / GENESIS G, J / Q...",F,01.10.2022
760,91,1,Atlas-5(511)²,AV-084,5S,CC SLC-41,21.01.2022,GSSAP 5 (USA 324) / GSSAP 6 (USA 325),AV-084,CC SLC-41,GSSAP 5 (USA 324) / GSSAP 6 (USA 325),Ok,21.01.2022


In [161]:
df_6=dataframes[6]

In [264]:
pattern = r'^(?:\d{2}\.\d{2}\.\d{4}(?: [A-Z])?|(?:[A-Z] )?\d{2}\.\d{2}\.\d{4}|(?:\d{2}\.\d{2}\.\d{4}\.)|\d{2}\.\d{2}\.\d{4})(?: [a-z])?(?: \.)?$'
pattern_serial = r'^(?![0-9]{0,2}$)([A-Z0-9-/]{3,}|[A-Z0-9-\s)(]{2,}\([A-Z]{2}\)|[A-Za-z]{2}[0-9]{5}\-[0-9]{3}|[A-Z0-9/.]+\.[0-9/.])$'
pattern_ls= r'^(?:[A-Z]{2}|[A-Za-z]{2}\s[A-Za-z0-9-/]+|[A-Za-z]{2}|[A-Z]\s[A-Za-z]{0,3}\s[A-Z0-9-]+|[A-Za-z]{3}\s[A-Z0-9-]+)$'

In [183]:
for column in df_6.columns:
    df_6.loc[df_6[column].str.contains(pattern), 'date'] = df_6.loc[df_6[column].str.contains(pattern), column]

In [265]:
df_6.drop(columns = 'ls', inplace = True)

In [182]:
for column in df_6.columns:
    df_6[column]=df_6[column].str.strip()

df_6['date'] = df_6['date'].fillna('00.00.2022')

for column in range(4):  
    df_6.loc[df_6[column].str.contains(pattern_serial) , 'serial'] = df_6.loc[df_6[column].str.contains(pattern_serial) , column]

In [266]:
for column in range(4,6): 
    df_6.loc[df_6[column].str.contains(pattern_ls), 'ls'] =  df_6.loc[df_6[column].str.contains(pattern_ls)  , column]

In [267]:
df_6[:60]

,0,1,2,3,4,5,6,date,serial,ls
0,2,TV-3,1,Vanguard,06.12.1957 F,CC LC-18A,Vanguard (Test Satellite F),06.12.1957 F,TV-3,CC LC-18A
1,3,TV-3BU,2,Vanguard,05.02.1958 F,CC LC-18A,Vanguard (Test Satellite G),05.02.1958 F,TV-3BU,CC LC-18A
2,4,TV-4,3,Vanguard,17.03.1958,CC LC-18A,Vanguard 1 (Test Satellite H),17.03.1958,TV-4,CC LC-18A
3,5,TV-5,4,Vanguard,29.04.1958 F,CC LC-18A,Vanguard (X-ray/Environmental Satellite),29.04.1958 F,TV-5,CC LC-18A
4,6,SLV-1,5,Vanguard,28.05.1958 F,CC LC-18A,Vanguard (Lyman-Alpha Satellite 1),28.05.1958 F,SLV-1,CC LC-18A
5,7,SLV-2,6,Vanguard,26.06.1958 F,CC LC-18A,Vanguard (Lyman-Alpha Satellite 2),26.06.1958 F,SLV-2,CC LC-18A
6,8,SLV-3,7,Vanguard,26.09.1958 F,CC LC-18A,Vanguard (Cloud Cover Satellite 1),26.09.1958 F,SLV-3,CC LC-18A
7,9,SLV-4,8,Vanguard,17.02.1959,CC LC-18A,Vanguard 2 (Cloud Cover Satellite 2),17.02.1959,SLV-4,CC LC-18A
8,10,SLV-5,9,Vanguard,14.04.1959 F,CC LC-18A,Vanguard (Magnetometer Satellite) / Vanguard B...,14.04.1959 F,SLV-5,CC LC-18A
9,11,SLV-6,10,Vanguard,22.06.1959 F,CC LC-18A,Vanguard (Radiation Balance Satellite),22.06.1959 F,SLV-6,CC LC-18A


In [268]:
df_6[df_6['ls'].isna()]

,0,1,2,3,4,5,6,date,serial,ls
403,29,29,Voskhod,N15001-03,27.08.1966,Ba LC-1/5 ?,Kosmos 128 (Zenit-4 #22),27.08.1966,N15001-03,NaN
679,3,1,Atlas-LV3C Centaur-C,AC-,3,30.06.1964 F CC LC-36A,Centaur AC-3,00.00.2022,AC-,NaN
680,4,2,Atlas-LV3C Centaur-C,AC-,4,11.12.1964 P CC LC-36A,Surveyor-Model 1,00.00.2022,AC-,NaN
681,5,3,Atlas-LV3C Centaur-C,AC-,5,02.03.1965 F CC LC-36A,Surveyor-SD 1,00.00.2022,AC-,NaN
863,195,5,Molniya-M (Blok-L),N15000-052,24.08.1966,Ba LC-31/6 ?,Luna 11 (Ye-6LF №101),24.08.1966,N15000-052,NaN
980,4,4,R-36-O,17.09.1966,f,Ba LC-162/36 OGCh #5,U 1,17.09.1966,R-36-O,NaN
981,5,5,R-36-O,02.11.1966,f,Ba LC-162/36 OGCh #6,U 2,02.11.1966,R-36-O,NaN
1033,24,19,Proton-K Blok-D,247-01,06.02.1970 F,Ba LC-81/23?,Luna (16) (Ye-8-5 №405),06.02.1970 F,247-01,NaN
1225,1385,125,Molniya-M (Blok-ML),77011-691,15.02.1989,Ba LC-1/5 ?,Molniya-1T 75 (Molniya-1T 84L),15.02.1989,77011-691,NaN
1292,66,14,Proton-K,292-01,30.03.1978,Ba LC-81/24?,Kosmos 997 (TKS-VA #5) / Kosmos 998 (TKS-VA #6),30.03.1978,292-01,NaN


In [167]:
buscador_links('thor')

https://space.skyrocket.de/doc_lau_det/thor-dm18_able-1.htm
https://space.skyrocket.de/doc_lau_det/thor-dm18_agena-a.htm
https://space.skyrocket.de/doc_lau_det/thor-dm18_able-3.htm
https://space.skyrocket.de/doc_lau_det/thor-dm18_able-2m.htm
https://space.skyrocket.de/doc_lau_det/thor-dm18_able-4.htm
https://space.skyrocket.de/doc_lau_det/thor-dm21_ablestar.htm
https://space.skyrocket.de/doc_lau_det/thor-dm21_agena-b.htm
https://space.skyrocket.de/doc_lau_det/thor-dm21_agena-d.htm
https://space.skyrocket.de/doc_lau_det/thor-slv2a_agena-d.htm
https://space.skyrocket.de/doc_lau_det/thor-slv2a_agena-b.htm
https://space.skyrocket.de/doc_lau_det/thor-dsv2a_ablestar.htm
https://space.skyrocket.de/doc_lau_det/thor-lv2d_burner-1-1.htm
https://space.skyrocket.de/doc_lau_det/thor-lv2d_burner-1-2.htm
https://space.skyrocket.de/doc_lau_det/thorad-slv2g_agena-d.htm
https://space.skyrocket.de/doc_lau_det/thor-lv2f_burner-2.htm
https://space.skyrocket.de/doc_lau_det/thor-lv2f_burner-2_star-13a.htm
ht

In [533]:
buscador_links_sat('luna')

https://space.skyrocket.de/doc_sdat/luna_e1.htm
https://space.skyrocket.de/doc_sdat/luna_e1.htm
https://space.skyrocket.de/doc_sdat/luna_e1.htm
https://space.skyrocket.de/doc_sdat/luna_e1.htm
https://space.skyrocket.de/doc_sdat/luna_e1a.htm
https://space.skyrocket.de/doc_sdat/luna_e1a.htm
https://space.skyrocket.de/doc_sdat/luna_e2a.htm
https://space.skyrocket.de/doc_sdat/luna_e3.htm
https://space.skyrocket.de/doc_sdat/luna_e3.htm
https://space.skyrocket.de/doc_sdat/luna_e6.htm
https://space.skyrocket.de/doc_sdat/luna_e6.htm
https://space.skyrocket.de/doc_sdat/luna_e6.htm
https://space.skyrocket.de/doc_sdat/luna_e6.htm
https://space.skyrocket.de/doc_sdat/luna_e6.htm
https://space.skyrocket.de/doc_sdat/luna_e6.htm
https://space.skyrocket.de/doc_sdat/luna_e6.htm
https://space.skyrocket.de/doc_sdat/luna_e6.htm
https://space.skyrocket.de/doc_sdat/luna_e6.htm
https://space.skyrocket.de/doc_sdat/luna_e6.htm
https://space.skyrocket.de/doc_sdat/lunar-orbiter.htm
https://space.skyrocket.de/doc_